In [34]:
import sys
sys.path.append('..')

import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import pandas as pd
import graph_tool as gt
from tqdm import tqdm
from operator import itemgetter
from functools import reduce
import itertools
import scipy.sparse as sps
import random
import os
#from pymnet import *

import MuxVizPy as mxp

import gseapy as gp

import warnings
warnings.filterwarnings("ignore")


#set.seed(1)

# input data settings
NEIGH_ORDER = 1 # or 0, order of nerighbors, 0 only connected proteins, 1 also first neighbors
CUT_THR = 0.7   # don't change this one

target_folder = "../Data/Virus_data_Enriched_"+str(CUT_THR)+"_Neigh_"+str(NEIGH_ORDER)+"/"

# multilayer settings
layerCouplingStrength = 1
networkOfLayersType = "categorical" ## = all to all

#virus metadata
virus_metadata = pd.read_csv("../Data/Files/viruses_metadata.csv", header=0, sep=";")

virus_metadata_onco = virus_metadata[virus_metadata["isOncogenic"] == True].reset_index()
virus_metadata_nonco = virus_metadata[virus_metadata["isOncogenic"] == False].reset_index()

#dictionary containing a unquie mapping between name of the protein and a corresponding index
node_map_df = pd.read_csv("../Data/Files/node_map.csv")
node_map_dict = {k:(v-1) for k,v in zip(node_map_df["Prot"], node_map_df["Index"])}

#function to create list of n_iter combination of nonco virus indexes with a fixed random seed for repitibility
def SamplingForNoco(n_iter, start=0, group_dim=8, random_seed=1234):
    np.random.seed(random_seed)
    nonco_cond = np.where(np.all([np.array(virus_metadata["virus"]!="Human_SARS_coronavirus_2"),
                                  np.array(virus_metadata["virus_short"]!="Lymphocytic_choriomeningitis_virus"),
                                  np.array(virus_metadata["neigh_order"]==NEIGH_ORDER), 
                                  np.array(virus_metadata["isOncogenic"]==False)],
                                  axis=0))
    
    nonco_sampling = np.array([np.random.choice(nonco_cond[0], group_dim, replace=False) for i in range(n_iter+start)])
    return nonco_sampling[start:(n_iter+start)]

In [2]:
onco_virus_indexes = np.where(np.array(np.all([virus_metadata["neigh_order"]==NEIGH_ORDER, virus_metadata["isOncogenic"] == True], axis=0)))[0]
net_onco = mxp.VirusMultiplex(onco_virus_indexes, target_folder=target_folder, virus_metadata=virus_metadata)

for v in net_onco.virus_list:
    print(v)
    
print("\n"+net_onco.net_description)

Epstein-Barr_virus
Hepatitis_B_virus_genotype_C_subtype_ayr
Hepatitis_C_virus_genotype_1a
Human_herpesvirus_8_type_P
Human_papillomavirus_type_16
Human_papillomavirus_type_18
Human_papillomavirus_type_5
Human_T-cell_leukemia_virus_1

Multiplex with 8 layers, 11873 nodes and 2208041 edges


In [3]:
tensor_onco = mxp.build.get_node_tensor_from_network_list(net_onco.g_list)
layer_tensor_onco = mxp.build.build_layers_tensor(Layers=net_onco.Layers,
                                                  OmegaParameter=layerCouplingStrength, 
                                                  MultisliceType=networkOfLayersType
                                                  )

supra_onco = mxp.build.build_supra_adjacency_matrix_from_edge_colored_matrices(nodes_tensor=tensor_onco,
                                                                               layer_tensor=layer_tensor_onco,
                                                                               layers=net_onco.Layers,
                                                                               nodes=net_onco.Nodes)


In [4]:
def supra_transition_matrix_virus(supra, node_tensor, nodes, layers, p_intra = 1):
    
    supra_sum = np.array(supra.sum(axis=0).tolist()[0])
    supra_nonz = supra.nonzero()
    
    blocks = []
    norms=[]

    #create the non_diagonal blocks: the diagonal entries are the number of non non-itrelayer connections of the nodes he can reach
    for l in range(layers):
        # i subtract (layers-1) to delete the contributions of the interlayer connections given by the categorical coupling
        block = sps.identity(nodes).multiply(supra_sum[l*nodes:(l+1)*nodes]-(layers-1))
        blocks.append(block)
    
    #the blocks are combined together and the rows are normalized to create a transition matrix without diagonal blocks
    mat =[]
    for la in range(layers):
        norm_fac = np.sum(supra_sum.reshape(layers, nodes)[np.delete(np.arange(layers),la)]-(layers-1), axis=0)
        norm_fac = np.where(norm_fac != 0, 1 / norm_fac, 0)
        mat.append([blocks[i].multiply(norm_fac) for i in np.delete(np.arange(layers), la)])
    
    #creating diagonal blocks by applying the standard prodecure for trans matr to each adj matrix
    diag_blocks = []
    for la in range(layers):
        t0_sum = np.array(list(node_tensor[la].sum(axis=0)))[0][0]
        t0_sum = np.where(t0_sum != 0, 1 / t0_sum, 0)

        diag_blocks.append(node_tensor[la].dot(sps.diags(t0_sum)).T)
    
    #combine the results in the unnormalized final matrix
    #the sum of each row can be 2, if there are contribution both from intra e inter connections, 1 for only intra, and 0 for nodes that becomes disconnected
    #if I want to give an exctra contribution to the intra connections, i should choose p_intra>1
    for i in range(layers):
        mat[i].insert(i, diag_blocks[i].multiply(np.array([p_intra]*nodes)))

    comb_mat =sps.vstack([sps.hstack(mat[i]) for i in range(layers)])
    
    valss = np.where(comb_mat.sum(axis=1) != 0, 1 / comb_mat.sum(axis=1), 0).flatten()

    return comb_mat.T.multiply(valss).T

# Enr Data

In [5]:
#O1S
comb = list(itertools.combinations(range(8), 7))
Sars_onco_cond = np.where(np.array(np.all([virus_metadata["neigh_order"]==NEIGH_ORDER, virus_metadata["isOncogenic"] == True], axis=0)))[0]
Sars_pos = np.where(np.array(np.all([virus_metadata["neigh_order"]==NEIGH_ORDER, virus_metadata["virus"]=="Human_SARS_coronavirus_2"], axis=0)))[0][0]
o1s_virus_indexes = np.array([list(Sars_onco_cond[list(comb[i])])+[Sars_pos] for i in range(len(comb))])

#N1S
Snonco_nonco_samples = SamplingForNoco(200, group_dim=7, random_seed=4563)
Sars_pos = np.where(np.array(np.all([virus_metadata["neigh_order"]==NEIGH_ORDER, virus_metadata["virus"]=="Human_SARS_coronavirus_2"], axis=0)))[0][0]
n1s_virus_indexes = np.concatenate([Snonco_nonco_samples, np.repeat(Sars_pos,200).reshape([200,1])], axis=1)

#N1O
n1o_virus_indexes = []
n1o_sampling = SamplingForNoco(200, group_dim=7, random_seed=456)
for i in range(len(n1o_sampling)):
    onco_pick = np.random.choice(onco_virus_indexes)
    n1o_virus_indexes.append(np.concatenate([n1o_sampling[i], [onco_pick]]))

#N2O
n2o_virus_indexes = []
n2o_sampling = SamplingForNoco(200, group_dim=6, random_seed=17521)
for i in range(len(n2o_sampling)):
    onco_pick = np.random.choice(onco_virus_indexes, 2)
    n2o_virus_indexes.append(np.concatenate([n2o_sampling[i], onco_pick]))
    
#N3O
n3o_virus_indexes = []
n3o_sampling = SamplingForNoco(200, group_dim=5, random_seed=27521)
for i in range(len(n3o_sampling)):
    onco_pick = np.random.choice(onco_virus_indexes, 3)
    n3o_virus_indexes.append(np.concatenate([n3o_sampling[i], onco_pick]))

#N4O
n4o_virus_indexes = []
n4o_sampling = SamplingForNoco(200, group_dim=4, random_seed=796581)
for i in range(len(n4o_sampling)):
    onco_pick = np.random.choice(onco_virus_indexes, 4)
    n4o_virus_indexes.append(np.concatenate([n4o_sampling[i], onco_pick]))    

#N5O
n5o_virus_indexes = []
n5o_sampling = SamplingForNoco(200, group_dim=3, random_seed=37521)
for i in range(len(n5o_sampling)):
    onco_pick = np.random.choice(onco_virus_indexes, 5)
    n5o_virus_indexes.append(np.concatenate([n5o_sampling[i], onco_pick]))
    
#N6O
n6o_virus_indexes = []
n6o_sampling = SamplingForNoco(200, group_dim=2, random_seed=47521)
for i in range(len(n6o_sampling)):
    onco_pick = np.random.choice(onco_virus_indexes, 6)
    n6o_virus_indexes.append(np.concatenate([n6o_sampling[i], onco_pick]))
    
#O1N
comb = list(itertools.combinations(range(8), 7))
o1n_sampling = SamplingForNoco(200, group_dim=1, random_seed=12541).flatten()
o1n_virus_indexes = []
for i in range(len(onco_virus_indexes)):
    onco_samp = onco_virus_indexes[list(comb[i])]
    for j in range(len(o1n_sampling)):
        o1n_virus_indexes.append(np.concatenate([onco_samp, [o1n_sampling[j]]]))
        
o1n_virus_indexes = random.sample(o1n_virus_indexes, 200)
        
#N
n_virus_indexes = SamplingForNoco(200, random_seed=41252145)

In [22]:
index_lists = [o1s_virus_indexes, 
               n1s_virus_indexes, 
               n1o_virus_indexes, 
               n2o_virus_indexes, 
               n3o_virus_indexes,
               n4o_virus_indexes, 
               n5o_virus_indexes, 
               n6o_virus_indexes, 
               o1n_virus_indexes, 
               n_virus_indexes]

folder_data = "../Data/EnrData/"
index_names = ["o1s", "n1s", "n1o", "n2o", "n3o", "n4o", "n5o", "n6o", "o1n", "n"]

In [28]:
if not os.path.isdir("../Data/EnrData/n"):
    os.mkdir("../Data/EnrData/n")

In [ ]:
for i in range(100):
    print("Iteration :", i+1)
    for j in tqdm(range(len(index_lists))):
        net = mxp.VirusMultiplex(index_lists[j][i], target_folder=target_folder, virus_metadata=virus_metadata)
        tensor = mxp.build.get_node_tensor_from_network_list(net.g_list)
        layer_tensor = mxp.build.build_layers_tensor(Layers=net.Layers,
                                                     OmegaParameter=layerCouplingStrength, 
                                                     MultisliceType=networkOfLayersType
                                                     )
        

        supra = mxp.build.build_supra_adjacency_matrix_from_edge_colored_matrices(nodes_tensor=tensor,
                                                                               layer_tensor=layer_tensor,
                                                                               layers=net.Layers,
                                                                               nodes=net.Nodes)


        def_mat = supra_transition_matrix_virus(supra=supra,
                                                node_tensor = tensor,
                                                nodes=net.Nodes,
                                                layers=net.Layers,
                                                p_intra=1)
        
        

        l_eig = mxp.leading_eigenv_approx.leading_eigenv_approx(def_mat.T)[1]

        res_df = pd.DataFrame({"rep_node": np.arange(def_mat.shape[0]),
                      "phy_node": np.arange(def_mat.shape[0])-((np.arange(def_mat.shape[0])//net.Nodes)*net.Nodes),
                      "layer": np.arange(def_mat.shape[0])//net.Nodes,
                      "prob": l_eig})

        agg_res = res_df.groupby("phy_node").aggregate(sum).sort_values("prob", ascending=False)

        list_res = np.array(list(net.node_map.keys()))[agg_res.index[:50]]

        np.savetxt(X=list_res, fname="list_res.txt", fmt="%s")

        enr_lvc = gp.enrichr(gene_list="list_res.txt", # or "./tests/data/gene_list.txt",
                             gene_sets=['KEGG_2021_Human'],
                             organism='human', # don't forget to set organism to the one you desired! e.g. Yeast
                             outdir=None, # don't write to disk
                            )
        if not os.path.isdir(folder_data+index_names[j]):
            os.mkdir(folder_data+index_names[j])
        enr_lvc.results.sort_values("Adjusted P-value")[["Term", "Adjusted P-value", "Combined Score"]].to_csv(folder_data+index_names[j]+"/"+str(i)+".csv", index=False)

Iteration : 1


100%|███████████████████████████████████████████| 10/10 [01:14<00:00,  7.40s/it]


Iteration : 2


100%|███████████████████████████████████████████| 10/10 [01:15<00:00,  7.52s/it]


Iteration : 3


100%|███████████████████████████████████████████| 10/10 [01:19<00:00,  7.90s/it]


Iteration : 4


100%|███████████████████████████████████████████| 10/10 [01:16<00:00,  7.64s/it]


Iteration : 5


 90%|███████████████████████████████████████▌    | 9/10 [01:13<00:08,  8.79s/it]

In [21]:
enr_lvc.results.sort_values("Adjusted P-value")